In [ ]:
import sys
sys.path.append("./notebooks")
sys.path.append("./src")

In [ ]:
import numpy as np

In [ ]:
data = np.load("../DATA/dataMonthly.npy")

In [ ]:
data.shape

### band indices:
* 0: Soil-Adjusted-Vegetation-Index; useful in dry/sparse regions
* 1: Enhanced-Vegetation-Index; Dense Forests (corrects soil and athmosphere noise)
* 2: Normalized-Difference-Red-Edge(NDRE) 705; early Stress
* 3: NDRE740; balanced stress
* 4: NDRE783; heavy stress
* 5: Normalized-Difference-Vegetation-Index; Vegetation Health (Greenness)
* 6: Normalized-Difference-Water-Index GAO; Waterstress
* 7: Normalized-Difference-Water-Index McFeeters; seperating land vs water
* 8: Normalized-Burn-Ratio; Burned Area/Forest Disturbance

In [ ]:
data.shape

In [ ]:
import copy

data_mask = data == 0
data_m = copy.deepcopy(data)

# Manual neighbor averaging along axis 0
neighbor_means = np.zeros_like(data, dtype=float)
neighbor_means[1:-1] = (data[:-2] + data[2:]) / 2
neighbor_means[0] = data[1]  # Edge case
neighbor_means[-1] = data[-2]  # Edge case

data_m[data_mask] = neighbor_means[data_mask]

In [ ]:
example_pixel = data_m[:, :, 100, 800]

In [ ]:
example_pixel.shape

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

In [ ]:
x = np.arange(0, 69)
y = pd.Series(example_pixel[:, 1])
yd = y.diff(12)
    
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=y))
fig.add_trace(go.Scatter(x=x, y=yd))
fig.show()

In [ ]:
for i in range(9):
    x = np.arange(0, 69)
    y = example_pixel[:, i]
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=x, y=y))
    fig.show()